### OpenSearch - Hybrid Search 

I have taken LangChain's OpenSearch integration to the next level by adding hybrid search capabilities. Building on the existing OpenSearchVectorSearch class, I have implemented Hybrid Search functionality (which combines the best of both keyword and semantic search). This new functionality allows users to harness the power of OpenSearch's advanced hybrid search features without leaving the familiar LangChain ecosystem. By blending traditional text matching with vector-based similarity, the enhanced class delivers more accurate and contextually relevant results. It's designed to seamlessly fit into existing LangChain workflows, making it easy for developers to upgrade their search capabilities.

In implementing hybrid search for OpenSearch within the LangChain framework, I also incorporated filtering capabilities. It's important to note that according to the OpenSearch hybrid search documentation, only post-filtering is supported for hybrid queries. This means that the filtering is applied after the hybrid search results are obtained, rather than during the initial search process.

**Note:** For the implementation of hybrid search, I strictly followed the official OpenSearch Hybrid search documentation. 

Importing Required Libraries

In [2]:
from opensearch_vector_search import OpenSearchVectorSearch
from langchain_openai import OpenAIEmbeddings
import os

Loading all environment variables

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

#### Embedding model 

In [4]:
def get_embedding_model(model_name: str):
    return OpenAIEmbeddings(
        model=model_name
    )

embedding_model = get_embedding_model(model_name="text-embedding-ada-002")

#### Instantiate OpenSearchVectorSearch vector store

In [5]:
opensearch_vectorstore = OpenSearchVectorSearch(
    index_name=os.getenv("INDEX_NAME"),
    embedding_function=embedding_model,
    opensearch_url=os.getenv("OPENSEARCH_URL"),
    http_auth= (os.getenv("OPENSEARCH_USERNAME"),os.getenv("OPENSEARCH_PASSWORD")),
    use_ssl=False,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False
)

### Creating Search Pipeline 

In [7]:
opensearch_vectorstore.create_search_pipelines(
    pipeline_name="search_pipeline_keyword_0.3_vector_0.7",
    keyword_weight=0.3,
    vector_weight=0.7,
)

<Response [200]>

### Hybrid Search

In [16]:
query = "what are the country named in our database?"

top_k = 3

pipeline_name = "search_pipeline_keyword_0.3_vector_0.7"

matched_docs = opensearch_vectorstore.similarity_search_with_score(
                query=query,
                k=top_k,
                search_type="hybrid_search",
                search_pipeline = pipeline_name,
                vector_field="vector_field",
                text_field="text",
                metadata_field="metadata"
            )

matched_docs

[(Document(page_content='Italy,[a] officially the Italian Republic,[b] is a country in Southern[12] and Western[13][c] Europe. It is located on a peninsula that extends into the middle of the Mediterranean Sea, with the Alps on its northern land border, as well as islands, notably Sicily and Sardinia.[15] Italy shares its borders with France, Switzerland, Austria, Slovenia and two enclaves: Vatican City and San Marino. It', metadata={'date': '20240916', 'parent_id': 'doc345', 'name': 'Italy', 'source': 'https://api.python.langchain.com/', 'published': True, 'lang': 'eng'}),
  0.7),
 (Document(page_content='and two enclaves: Vatican City and San Marino. It is the tenth-largest country in Europe, covering an area of 301,340 km2 (116,350 sq mi),[3] and third-most populous member state of the European Union, with a population of nearly 60 million.[16] Its capital and largest city is Rome; other major urban areas include Milan, Naples, Turin, Florence, and Venice.In antiquity, Italy was hom

In [19]:
def pretty_print(matched_docs: list):
    for doc,score in matched_docs: 
        print(f"Content: {doc.page_content}")
        print(f"Metadata: {doc.metadata}")
        print(f"Score: {score}")
        print("\n")

In [20]:
pretty_print(matched_docs)

Content: Italy,[a] officially the Italian Republic,[b] is a country in Southern[12] and Western[13][c] Europe. It is located on a peninsula that extends into the middle of the Mediterranean Sea, with the Alps on its northern land border, as well as islands, notably Sicily and Sardinia.[15] Italy shares its borders with France, Switzerland, Austria, Slovenia and two enclaves: Vatican City and San Marino. It
Metadata: {'date': '20240916', 'parent_id': 'doc345', 'name': 'Italy', 'source': 'https://api.python.langchain.com/', 'published': True, 'lang': 'eng'}
Score: 0.7


Content: and two enclaves: Vatican City and San Marino. It is the tenth-largest country in Europe, covering an area of 301,340 km2 (116,350 sq mi),[3] and third-most populous member state of the European Union, with a population of nearly 60 million.[16] Its capital and largest city is Rome; other major urban areas include Milan, Naples, Turin, Florence, and Venice.In antiquity, Italy was home to numerous
Metadata: {'date

#### Experiment - 1 




#### Hybrid Search 

Keyword_weight: 1, vector_weight: 0 

I conducted an experiment to verify the accuracy of my hybrid search implementation by comparing it to a plain keyword search. For this test, I set the keyword_weight to 1 and the vector_weight to 0 in the hybrid search, effectively giving full weightage to the keyword component. The results from this hybrid search configuration matched those of a plain keyword search, confirming that my implementation can accurately reproduce keyword-only search results when needed. It's important to note that while the results were the same, the scores differed between the two methods. This difference is expected because the plain keyword search in OpenSearch uses the BM25 algorithm for scoring, whereas the hybrid search still performs both keyword and vector searches before normalizing the scores, even when the vector component is given zero weight. This experiment validates that my hybrid search solution correctly handles the keyword search component and properly applies the weighting system, demonstrating its accuracy and flexibility in emulating different search scenarios.

In [30]:
opensearch_vectorstore.create_search_pipelines(
    pipeline_name="search_pipeline_keyword_1_vector_0",
    keyword_weight=1.0,
    vector_weight=0.0,
)

<Response [200]>

In [22]:
query = "what are the country named in our database?"

top_k = 3

pipeline_name = "search_pipeline_keyword_1_vector_0"

matched_docs = opensearch_vectorstore.similarity_search_with_score(
                query=query,
                k=top_k,
                search_type="hybrid_search",
                search_pipeline = pipeline_name,
                vector_field="vector_field",
                text_field="text",
                metadata_field="metadata"
            )

matched_docs

[(Document(page_content='A vector database, vector store or vector search engine is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms,[1][2] so that one can search the database with a query vector to retrieve the closest matching database records.Vectors are mathematical', metadata={'date': '2022-06-01', 'parent_id': 'doc653', 'name': 'Vector Store', 'source': 'https://api.python.langchain.com/', 'published': False, 'lang': 'eng'}),
  1.0),
 (Document(page_content="database records.Vectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, with the number of dimensions ranging from few hundreds to tens of thousands, depending on the complexity of the data being represented. A vector's position in this space represents its characteristics. Words, phrases, or entir

In [23]:
pretty_print(matched_docs)

Content: A vector database, vector store or vector search engine is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms,[1][2] so that one can search the database with a query vector to retrieve the closest matching database records.Vectors are mathematical
Metadata: {'date': '2022-06-01', 'parent_id': 'doc653', 'name': 'Vector Store', 'source': 'https://api.python.langchain.com/', 'published': False, 'lang': 'eng'}
Score: 1.0


Content: database records.Vectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, with the number of dimensions ranging from few hundreds to tens of thousands, depending on the complexity of the data being represented. A vector's position in this space represents its characteristics. Words, phrases, or entire
Metadata: {'date': '2022-06-

#### Plain Keyword Search

In [28]:
query = "what are the country named in our database?"

top_k = 3

matched_docs = opensearch_vectorstore.similarity_search_with_score(
                query=query,
                k=top_k,
                search_type="full_text_search",
                vector_field="vector_field",
                text_field="text",
                metadata_field="metadata"
            )

matched_docs

[(Document(page_content='A vector database, vector store or vector search engine is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms,[1][2] so that one can search the database with a query vector to retrieve the closest matching database records.Vectors are mathematical', metadata={'date': '2022-06-01', 'parent_id': 'doc653', 'name': 'Vector Store', 'source': 'https://api.python.langchain.com/', 'published': False, 'lang': 'eng'}),
  6.866866),
 (Document(page_content="database records.Vectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, with the number of dimensions ranging from few hundreds to tens of thousands, depending on the complexity of the data being represented. A vector's position in this space represents its characteristics. Words, phrases, or 

In [29]:
pretty_print(matched_docs)

Content: A vector database, vector store or vector search engine is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms,[1][2] so that one can search the database with a query vector to retrieve the closest matching database records.Vectors are mathematical
Metadata: {'date': '2022-06-01', 'parent_id': 'doc653', 'name': 'Vector Store', 'source': 'https://api.python.langchain.com/', 'published': False, 'lang': 'eng'}
Score: 6.866866


Content: database records.Vectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, with the number of dimensions ranging from few hundreds to tens of thousands, depending on the complexity of the data being represented. A vector's position in this space represents its characteristics. Words, phrases, or entire
Metadata: {'date': '202

### Experiment - 2

#### Hybrid Search 

keyword_weight = 0.0, vector_weight = 1.0

For experiment-2, I took the inverse approach to further validate my hybrid search implementation. I set the keyword_weight to 0 and the vector_weight to 1, effectively giving full weightage to the vector search component (KNN search). I then compared these results with a pure vector search. The outcome was consistent with my expectations: the results from the hybrid search with these settings exactly matched those from a standalone vector search. This confirms that my implementation accurately reproduces vector search results when configured to do so. As with the first experiment, I observed that while the results were identical, the scores differed between the two methods. This difference in scoring is expected and can be attributed to the normalization process in hybrid search, which still considers both components even when one is given zero weight. This experiment further validates the accuracy and flexibility of my hybrid search solution, demonstrating its ability to effectively emulate pure vector search when needed while maintaining the underlying hybrid search structure.

In [31]:
opensearch_vectorstore.create_search_pipelines(
    pipeline_name="search_pipeline_keyword_0_vector_1",
    keyword_weight=0.0,
    vector_weight=1.0,
)

<Response [200]>

In [32]:
query = "what are the country named in our database?"

top_k = 3

pipeline_name = "search_pipeline_keyword_0_vector_1"

matched_docs = opensearch_vectorstore.similarity_search_with_score(
                query=query,
                k=top_k,
                search_type="hybrid_search",
                search_pipeline = pipeline_name,
                vector_field="vector_field",
                text_field="text",
                metadata_field="metadata"
            )

matched_docs

[(Document(page_content='Italy,[a] officially the Italian Republic,[b] is a country in Southern[12] and Western[13][c] Europe. It is located on a peninsula that extends into the middle of the Mediterranean Sea, with the Alps on its northern land border, as well as islands, notably Sicily and Sardinia.[15] Italy shares its borders with France, Switzerland, Austria, Slovenia and two enclaves: Vatican City and San Marino. It', metadata={'date': '20240916', 'parent_id': 'doc345', 'name': 'Italy', 'source': 'https://api.python.langchain.com/', 'published': True, 'lang': 'eng'}),
  1.0),
 (Document(page_content='and two enclaves: Vatican City and San Marino. It is the tenth-largest country in Europe, covering an area of 301,340 km2 (116,350 sq mi),[3] and third-most populous member state of the European Union, with a population of nearly 60 million.[16] Its capital and largest city is Rome; other major urban areas include Milan, Naples, Turin, Florence, and Venice.In antiquity, Italy was hom

In [33]:
pretty_print(matched_docs)

Content: Italy,[a] officially the Italian Republic,[b] is a country in Southern[12] and Western[13][c] Europe. It is located on a peninsula that extends into the middle of the Mediterranean Sea, with the Alps on its northern land border, as well as islands, notably Sicily and Sardinia.[15] Italy shares its borders with France, Switzerland, Austria, Slovenia and two enclaves: Vatican City and San Marino. It
Metadata: {'date': '20240916', 'parent_id': 'doc345', 'name': 'Italy', 'source': 'https://api.python.langchain.com/', 'published': True, 'lang': 'eng'}
Score: 1.0


Content: and two enclaves: Vatican City and San Marino. It is the tenth-largest country in Europe, covering an area of 301,340 km2 (116,350 sq mi),[3] and third-most populous member state of the European Union, with a population of nearly 60 million.[16] Its capital and largest city is Rome; other major urban areas include Milan, Naples, Turin, Florence, and Venice.In antiquity, Italy was home to numerous
Metadata: {'date

#### Pure Vector Search

In [34]:
query = "what are the country named in our database?"

top_k = 3

matched_docs = opensearch_vectorstore.similarity_search_with_score(
                query=query,
                k=top_k,
                search_type="approximate_search",
                vector_field="vector_field",
                text_field="text",
                metadata_field="metadata"
            )

matched_docs

[(Document(page_content='Italy,[a] officially the Italian Republic,[b] is a country in Southern[12] and Western[13][c] Europe. It is located on a peninsula that extends into the middle of the Mediterranean Sea, with the Alps on its northern land border, as well as islands, notably Sicily and Sardinia.[15] Italy shares its borders with France, Switzerland, Austria, Slovenia and two enclaves: Vatican City and San Marino. It', metadata={'name': 'Italy', 'source': 'https://api.python.langchain.com/', 'date': '20240916', 'lang': 'eng', 'published': True, 'parent_id': 'doc345'}),
  1.7577173),
 (Document(page_content='and two enclaves: Vatican City and San Marino. It is the tenth-largest country in Europe, covering an area of 301,340 km2 (116,350 sq mi),[3] and third-most populous member state of the European Union, with a population of nearly 60 million.[16] Its capital and largest city is Rome; other major urban areas include Milan, Naples, Turin, Florence, and Venice.In antiquity, Italy w

In [35]:
pretty_print(matched_docs)

Content: Italy,[a] officially the Italian Republic,[b] is a country in Southern[12] and Western[13][c] Europe. It is located on a peninsula that extends into the middle of the Mediterranean Sea, with the Alps on its northern land border, as well as islands, notably Sicily and Sardinia.[15] Italy shares its borders with France, Switzerland, Austria, Slovenia and two enclaves: Vatican City and San Marino. It
Metadata: {'name': 'Italy', 'source': 'https://api.python.langchain.com/', 'date': '20240916', 'lang': 'eng', 'published': True, 'parent_id': 'doc345'}
Score: 1.7577173


Content: and two enclaves: Vatican City and San Marino. It is the tenth-largest country in Europe, covering an area of 301,340 km2 (116,350 sq mi),[3] and third-most populous member state of the European Union, with a population of nearly 60 million.[16] Its capital and largest city is Rome; other major urban areas include Milan, Naples, Turin, Florence, and Venice.In antiquity, Italy was home to numerous
Metadata: 

### Experiment - 3

Hybrid Search - balanced 

keyword_weight = 0.5, vector_weight = 0.5

For experiment-3, I adopted a balanced approach to further evaluate the effectiveness of my hybrid search implementation. In this test, I set both the keyword_weight and vector_weight to 0.5, giving equal importance to keyword-based and vector-based search components. This configuration aims to leverage the strengths of both search methods simultaneously.
By setting both weights to 0.5, I intended to create a scenario where the hybrid search would consider lexical matches and semantic similarity equally. This balanced approach is often ideal for many real-world applications, as it can capture both exact keyword matches and contextually relevant results that might not contain the exact search terms.

In [36]:
opensearch_vectorstore.create_search_pipelines(
    pipeline_name="search_pipeline_keyword_0.5_vector_0.5",
    keyword_weight=0.5,
    vector_weight=0.5,
)

<Response [200]>

In [37]:
query = "what are the country named in our database?"

top_k = 3

pipeline_name = "search_pipeline_keyword_0.5_vector_0.5"

matched_docs = opensearch_vectorstore.similarity_search_with_score(
                query=query,
                k=top_k,
                search_type="hybrid_search",
                search_pipeline = pipeline_name,
                vector_field="vector_field",
                text_field="text",
                metadata_field="metadata"
            )

matched_docs

[(Document(page_content='Italy,[a] officially the Italian Republic,[b] is a country in Southern[12] and Western[13][c] Europe. It is located on a peninsula that extends into the middle of the Mediterranean Sea, with the Alps on its northern land border, as well as islands, notably Sicily and Sardinia.[15] Italy shares its borders with France, Switzerland, Austria, Slovenia and two enclaves: Vatican City and San Marino. It', metadata={'date': '20240916', 'parent_id': 'doc345', 'name': 'Italy', 'source': 'https://api.python.langchain.com/', 'published': True, 'lang': 'eng'}),
  0.5),
 (Document(page_content='A vector database, vector store or vector search engine is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms,[1][2] so that one can search the database with a query vector to retrieve the closest matching database records.Vectors are mathematical

## Hybrid Search along with Metadata Filtering

#### Example-1: 

Retrieve only top-k documents which are not yet published & related to this query: "what are the country named in our database?"

published == 'False'

Weightage: keyword_weight=0.5, vector_weight=0.5,

In [38]:
opensearch_vectorstore.create_search_pipelines(
    pipeline_name="search_pipeline_keyword_0.5_vector_0.5",
    keyword_weight=0.5,
    vector_weight=0.5,
)

<Response [200]>

In [39]:
query = "what are the country named in our database?"

top_k = 3

pipeline_name = "search_pipeline_keyword_0.5_vector_0.5"

matched_docs = opensearch_vectorstore.similarity_search_with_score(
                query=query,
                k=top_k,
                search_type="hybrid_search",
                search_pipeline = pipeline_name,
                vector_field="vector_field",
                text_field="text",
                metadata_field="metadata",
                post_filter= {"bool": {"filter": {"term": {"metadata.published": False}}}}
            )

matched_docs

[(Document(page_content='A vector database, vector store or vector search engine is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms,[1][2] so that one can search the database with a query vector to retrieve the closest matching database records.Vectors are mathematical', metadata={'date': '2022-06-01', 'parent_id': 'doc653', 'name': 'Vector Store', 'source': 'https://api.python.langchain.com/', 'published': False, 'lang': 'eng'}),
  0.5),
 (Document(page_content="database records.Vectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, with the number of dimensions ranging from few hundreds to tens of thousands, depending on the complexity of the data being represented. A vector's position in this space represents its characteristics. Words, phrases, or entir

In [40]:
pretty_print(matched_docs)

Content: A vector database, vector store or vector search engine is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms,[1][2] so that one can search the database with a query vector to retrieve the closest matching database records.Vectors are mathematical
Metadata: {'date': '2022-06-01', 'parent_id': 'doc653', 'name': 'Vector Store', 'source': 'https://api.python.langchain.com/', 'published': False, 'lang': 'eng'}
Score: 0.5


Content: database records.Vectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, with the number of dimensions ranging from few hundreds to tens of thousands, depending on the complexity of the data being represented. A vector's position in this space represents its characteristics. Words, phrases, or entire
Metadata: {'date': '2022-06-

Same Query but without any Metadata filtering

In [42]:
query = "what are the country named in our database?"

top_k = 3

pipeline_name = "search_pipeline_keyword_0.5_vector_0.5"

matched_docs = opensearch_vectorstore.similarity_search_with_score(
                query=query,
                k=top_k,
                search_type="hybrid_search",
                search_pipeline = pipeline_name,
                vector_field="vector_field",
                text_field="text",
                metadata_field="metadata"
            )

pretty_print(matched_docs)

Content: Italy,[a] officially the Italian Republic,[b] is a country in Southern[12] and Western[13][c] Europe. It is located on a peninsula that extends into the middle of the Mediterranean Sea, with the Alps on its northern land border, as well as islands, notably Sicily and Sardinia.[15] Italy shares its borders with France, Switzerland, Austria, Slovenia and two enclaves: Vatican City and San Marino. It
Metadata: {'date': '20240916', 'parent_id': 'doc345', 'name': 'Italy', 'source': 'https://api.python.langchain.com/', 'published': True, 'lang': 'eng'}
Score: 0.5


Content: A vector database, vector store or vector search engine is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms,[1][2] so that one can search the database with a query vector to retrieve the closest matching database records.Vectors are mathematical
Metadata: {'date': '2022-06-0

#### Example-2: 

Retrieve only top-k documents which are in italian language & related to this query: "what are the country named in our database?"

lang == 'ita'

Weightage: keyword_weight=0.5, vector_weight=0.5,

In [48]:
query = "what are the latest news?"

top_k = 3

pipeline_name = "search_pipeline_keyword_0.5_vector_0.5"

matched_docs = opensearch_vectorstore.similarity_search_with_score(
                query=query,
                k=top_k,
                search_type="hybrid_search",
                search_pipeline = pipeline_name,
                vector_field="vector_field",
                text_field="text",
                metadata_field="metadata",
                post_filter= {"bool": {"filter": {"term": {"metadata.lang": "ita"}}}}

            )


pretty_print(matched_docs)

Content: software continui. La smentita fa seguito alla reazione negativa sui commenti fatti dal CEO Hanneke Faber, che aveva accennato alla possibilità durante un'intervista podcast con The Verge.In risposta a questi rapporti, la responsabile delle comunicazioni di Logitech, Nicole Kenyon, ha chiarito: Non ci sono piani per un mouse in abbonamento. Questa dichiarazione è stata rilasciata a diversi media
Metadata: {'date': '2023-07-28', 'parent_id': 'doc513', 'name': 'L’ultimo aggiornamento', 'section': 'news', 'source': 'https://dailyhunt.com', 'published': True, 'lang': 'ita'}
Score: 1.0


Content: La smentita fa seguito alla reazione negativa sui commenti fatti dal CEO Hanneke Faber, che aveva accennato alla possibilità durante un'intervista podcast con The Verge.In risposta a questi rapporti, la responsabile delle comunicazioni di Logitech, Nicole Kenyon, ha chiarito: Non ci sono piani per un mouse in abbonamento. Questa dichiarazione è stata rilasciata a diversi media sulla scia d

#### Example-3

Retrieve only top-k documents which are coming from 'sports' section & related to this query: "what are the country named in our database?

section == sports

Weightage: keyword_weight=0.5, vector_weight=0.5

In [49]:
query = "what are the latest sports news today?"

top_k = 3

pipeline_name = "search_pipeline_keyword_0.5_vector_0.5"

matched_docs = opensearch_vectorstore.similarity_search_with_score(
                query=query,
                k=top_k,
                search_type="hybrid_search",
                search_pipeline = pipeline_name,
                vector_field="vector_field",
                text_field="text",
                metadata_field="metadata",
                post_filter= {"bool": {"filter": {"term": {"metadata.section": "sports"}}}}
            )


pretty_print(matched_docs)

Content: place in the morning of the first competition day. On the second competition day, wrestlers who have qualified for the finals and repechage are weighed in again.It is with regret that the Indian contingent shares news of the disqualification of Vinesh Phogat from the women’s wrestling 50kg class,” the Indian Olympic Association (IOA) said in a statement. “Despite the best efforts by the team
Metadata: {'date': '2024-08-07', 'parent_id': 'doc680', 'name': 'Why Vinesh Phogat was disqualified from Paris 2024 Olympics wrestling', 'section': 'sports', 'source': 'https://newstoday.com', 'published': True, 'lang': 'eng'}
Score: 0.5005


Content: statement. “Despite the best efforts by the team through the night, she weighed in a few grams over 50kg this morning.No further comments will be made by the contingent at this time. The Indian team requests you respect Vinesh’s privacy. It would like to focus on the competitions on hand.Vinesh Phogat was eligible for competition on the openi

#### Example-4

Retrieve only top-k documents related to this query: "what are the country named in our database? 

& (post filter)

which are published in between these dates from '2018-01-01' to '2021-01-01'

**Condition:** range: greater than: 2018-01-01 & less than: 2021-01-01

Weightage: keyword_weight=0.5, vector_weight=0.5

In [50]:


query = "what are the latest sports news today?"

top_k = 3

pipeline_name = "search_pipeline_keyword_0.5_vector_0.5"

matched_docs = opensearch_vectorstore.similarity_search_with_score(
                query=query,
                k=top_k,
                search_type="hybrid_search",
                search_pipeline = pipeline_name,
                vector_field="vector_field",
                text_field="text",
                metadata_field="metadata",
                post_filter= {"bool": 
                      {"filter": 
                       {"range": {"metadata.date": 
                                  {"gte": '2018-01-01', "lte": '2021-01-01'}}}}}
            )


pretty_print(matched_docs)

Content: of Helen telling him they need to go home. He breaks into a nearby animal clinic, treats his wounds, and adopts a pit bull puppy scheduled to be euthanized before beginning to walk home.
Metadata: {'date': '2020-09-16', 'parent_id': 'doc123', 'name': 'John wick: Chapter 1', 'source': 'https://johnwick.com/', 'published': True, 'lang': 'eng'}
Score: 0.5


Content: margins of the Indus river basin 9,000 years ago, evolving gradually into the Indus Valley Civilisation of the third millennium BCE.[32] By 1200 BCE, an archaic form of Sanskrit, an Indo-European language, had diffused into India from the northwest.[33][34] Its evidence today is found in the hymns of the Rigveda. Preserved by an oral tradition that was resolutely vigilant, the Rigveda records the
Metadata: {'date': '2018-05-19', 'parent_id': 'doc633', 'name': 'India', 'source': 'https://india.com/', 'published': True, 'lang': 'eng'}
Score: 0.5


Content: home, assault him, kill the puppy, and steal the car. Iosef take

#### Example-5 

Combining muliple metadata filtering

In [51]:


query = "what are the latest sports news today?"

top_k = 3

pipeline_name = "search_pipeline_keyword_0.5_vector_0.5"

matched_docs = opensearch_vectorstore.similarity_search_with_score(
                query=query,
                k=top_k,
                search_type="hybrid_search",
                search_pipeline = pipeline_name,
                vector_field="vector_field",
                text_field="text",
                metadata_field="metadata",
                post_filter= {"bool": 
                        {"filter": [
                        {"range": {"metadata.date": {"gte": '2018-01-01', "lte": '2021-01-01'}}},
                        {"term": {"metadata.published": True}}
                        ]
                        }}
                )

pretty_print(matched_docs)

Content: of Helen telling him they need to go home. He breaks into a nearby animal clinic, treats his wounds, and adopts a pit bull puppy scheduled to be euthanized before beginning to walk home.
Metadata: {'date': '2020-09-16', 'parent_id': 'doc123', 'name': 'John wick: Chapter 1', 'source': 'https://johnwick.com/', 'published': True, 'lang': 'eng'}
Score: 0.5


Content: margins of the Indus river basin 9,000 years ago, evolving gradually into the Indus Valley Civilisation of the third millennium BCE.[32] By 1200 BCE, an archaic form of Sanskrit, an Indo-European language, had diffused into India from the northwest.[33][34] Its evidence today is found in the hymns of the Rigveda. Preserved by an oral tradition that was resolutely vigilant, the Rigveda records the
Metadata: {'date': '2018-05-19', 'parent_id': 'doc633', 'name': 'India', 'source': 'https://india.com/', 'published': True, 'lang': 'eng'}
Score: 0.5


Content: home, assault him, kill the puppy, and steal the car. Iosef take